In [0]:
#noetbook utilizado para a aula sobre sistemas de recomendação

#1) Carregar os dados
#2) Pré-processamento dos dados
#3) Divisão dos dados entre treinamento e teste
#4) Criar o sistema de recomendação através do ALS
#5) Realizar previsões
#6) Testar o modelo

In [0]:
from pyspark.sql import SparkSession #importa a biblioteca que cria a seção do spark

In [0]:
#inicia a seção para a utilização do spark
spark = SparkSession.builder.appName("recomendacao").getOrCreate() #cria a seção caso não exista ou obtém a já criada

In [0]:
%fs ls /FileStore/tables

path,name,size
dbfs:/FileStore/tables/iris_bezdekIris.csv,iris_bezdekIris.csv,4551
dbfs:/FileStore/tables/regressaoLinear-1.csv,regressaoLinear-1.csv,564
dbfs:/FileStore/tables/regressaoLinear.csv,regressaoLinear.csv,564
dbfs:/FileStore/tables/u.data,u.data,1979173


In [0]:
diretorioRecomendacao="/FileStore/tables/u.data"  #diretório que contém o arquivo a ser utilizado

#1) Carregando o arquivo.

In [0]:
#lendo arquivos armazenados através da função genérica
rdd_movies = spark.sparkContext.textFile(diretorioRecomendacao)

In [0]:
rdd_movies.take(10)  #user id | item id | rating | timestamp

Out[5]: ['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596',
 '298\t474\t4\t884182806',
 '115\t265\t2\t881171488',
 '253\t465\t5\t891628467',
 '305\t451\t3\t886324817',
 '6\t86\t3\t883603013']

In [0]:
#definindo as bibliotecas a serem utilizadas
from pyspark.mllib.recommendation import ALS, Rating  #MLlib utilizada para implementar os algoritmos ALS e Rating

#2) Pré-processamento dos dados
#3) Separando os dados para treinamento e teste

In [0]:
#dividindo os dados entre treinamento e teste
(trainRatings, testRatings) = rdd_movies.randomSplit([0.7, 0.3])

In [0]:
trainRatings.take(5)

Out[8]: ['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596']

In [0]:
testRatings.first()  #print da primeira linha do RDD

Out[9]: '298\t474\t4\t884182806'

In [0]:
trainingData = trainRatings.map(lambda l: l.split('\t')).map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))  #aplicando a função Rating

In [0]:
trainingData.first()  #print do RDD criado através da função Rating

Out[11]: Rating(user=196, product=242, rating=3.0)

In [0]:
#mesmo procedimento para os dados de teste
testData = testRatings.map(lambda l: l.split('\t')).map(lambda l: (int(l[0]), int(l[1])))

In [0]:
testData.first()  #id do usuário | id do filme

Out[13]: (298, 474)

#4) Construindo o modelo ALS

In [0]:
#definindo as variáveis do modelo
rank = 10  # número de fatores latentes do modelo  R->P (usuários)*Q (itens) => R_mxn = P_mxrank * Q_rankxm  (em que m= numero de usuário e n= numero de itens)
numIterations = 50 #número de iterações realizadas pelo modelo
model = ALS.train(trainingData, rank, numIterations) # treina o modelo

#5) Previsão do modelo

In [0]:
model.predict(253, 465)  # entrada (usuário,produto)

Out[15]: 4.466731520836735

In [0]:
previsao = model.predictAll(testData)  #previsão para todos os dados de teste
previsao.first()

Out[16]: Rating(user=472, product=384, rating=3.9965990537611464)

In [0]:
previsao = previsao.map(lambda l: ((l[0], l[1]), l[2])) #mapea para a exibição
previsao.take(5)

Out[17]: [((472, 384), 3.9965990537611464),
 ((280, 384), 2.6927841914523922),
 ((880, 384), 3.0188828457584655),
 ((393, 384), 3.392364938759668),
 ((833, 384), 2.528360933526539)]

In [0]:
testRating2 = testRatings.map(lambda l: l.split('\t')).map(lambda l: ((int(l[0]), int(l[1])), float(l[2]))) # mapea para exibição e utilização na análise

In [0]:
testRating2.first()

Out[19]: ((298, 474), 4.0)

In [0]:
ratingsAndPredictions = testRating2.join(previsao)
ratingsAndPredictions.take(5)

Out[20]: [((305, 451), (3.0, 2.9779137217592515)),
 ((160, 234), (5.0, 4.28724201179358)),
 ((181, 1081), (1.0, 1.7234739417319214)),
 ((20, 288), (1.0, 1.412393237755367)),
 ((222, 750), (5.0, 4.644246991736601))]

#6) Avaliação do modelo

In [0]:
MSE = ratingsAndPredictions.map(lambda r: (r[1][0] - r[1][1])**2).mean()
print ("Erro médio quadrado = " + str(MSE))

Erro médio quadrado = 1.335228331364568

In [0]:
#dado um usuário -> recomenda os top 5 produtos
model.recommendProducts(105, 5) 

Out[23]: [Rating(user=105, product=1282, rating=6.9802682515362235),
 Rating(user=105, product=1192, rating=6.845899721258711),
 Rating(user=105, product=1220, rating=6.728303611709218),
 Rating(user=105, product=1058, rating=6.625006391780916),
 Rating(user=105, product=624, rating=6.549016118382394)]

In [0]:
#dado um produto -> recomenda top 5 usuários
model.recommendUsers(1, 5)  #filme Toy Story (1995)

Out[24]: [Rating(user=366, product=1, rating=7.010836974990126),
 Rating(user=688, product=1, rating=6.288551855539476),
 Rating(user=240, product=1, rating=6.200997788041424),
 Rating(user=810, product=1, rating=5.784221233304366),
 Rating(user=173, product=1, rating=5.641164166053034)]

In [0]:
#mostrando o vetor de características referentes usuários (coluna - P)
model.userFeatures().take(1)[0]

Out[25]: (8,
 array('d', [-1.0941311120986938, -0.21785837411880493, 0.6110883951187134, 0.5071765184402466, 0.019552892073988914, -0.47980913519859314, 0.06665246188640594, -0.8979180455207825, 1.6581037044525146, -0.21201744675636292]))

In [0]:
#mostra o vetor de características referente a um produto (linha - Q)
model.productFeatures().take(1)[0]

Out[26]: (8,
 array('d', [-0.5096912980079651, 1.1012227535247803, -0.44709670543670654, 1.311926245689392, -0.6736835241317749, -0.2042643129825592, 0.15809200704097748, -1.0532630681991577, 1.4660828113555908, -0.7111119627952576]))